Web scraping - Live code

In [ ]:
!pip install MechanicalSoup

In [ ]:
import mechanicalsoup
import pandas as pd
import sqlite3



In [ ]:
# create browser object & open URL
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://en.wikipedia.org/wiki/Comparison_of_Linux_distributions")



<Response [200]>

In [ ]:
# extract all table headers (entire "Distribution" column)
th = browser.page.find_all("th", attrs={"class": "table-rh"})


In [ ]:
# tidy up and slice off non-table elements
distribution = [value.text.replace("\n", "") for value in th]
distribution = distribution[:95]



In [ ]:
# extract table data (the rest of the table)
td = browser.page.find_all("td")


In [ ]:
# tidy up and slice off non-table elements
columns = [value.text.replace("\n", "") for value in td]
columns = columns[6:1051]

column_names = ["Founder",
                "Maintainer",
                "Initial_Release_Year",
                "Current_Stable_Version",
                "Security_Updates",
                "Release_Date",
                "System_Distribution_Commitment",
                "Forked_From",
                "Target_Audience",
                "Cost",
                "Status"]

dictionary = {"Distribution": distribution}



In [ ]:
# insert column names and their data into a dictionary
for idx, key in enumerate(column_names):
    dictionary[key] = columns[idx:][::11]



In [ ]:
# convert dictionary to data frame
df = pd.DataFrame(data = dictionary)



In [ ]:
# create new database and cursor
connection = sqlite3.connect("linux_distro.db")
cursor = connection.cursor()



In [ ]:
# create database table and insert all data frame rows
cursor.execute("create table linux (Distribution, " + ",".join(column_names)+ ")")
for i in range(len(df)):
    cursor.execute("insert into linux values (?,?,?,?,?,?,?,?,?,?,?,?)", df.iloc[i])



In [ ]:
# PERMANENTLY save inserted data in "linux_distro.db"
connection.commit()

connection.close()


In [ ]:
df

,Distribution,Founder,Maintainer,Initial_Release_Year,Current_Stable_Version,Security_Updates,Release_Date,System_Distribution_Commitment,Forked_From,Target_Audience,Cost,Status
0,AlmaLinux,AlmaLinux Foundation,AlmaLinux Foundation,2021,8.8[2],10 years[3],2023-05-18,X,Red Hat Enterprise Linux (RHEL),"server, workstation",None,Active
1,Alpine Linux,Alpine Linux Team,Alpine Linux Team,2006,3.18.3[4],?,2023-08-07,X,LEAF Project,"security, lightweight, general",None,Active
2,ALT Linux,ALT Linux Team,"ALT Linux Team, ALT Linux LLC",2001,10.2[5],?,2023-09-19,X,Mandrake Linux,"general, school",None,Active
3,antiX,Anticapitalista,Anticapitalista,2007,23[6],?,2023-08-28,X,"Debian, MEPIS",old computers,None,Active
4,ArchBang,Willensky Aristide,Stan McLaren,2011,Rolling,Rolling,Rolling,X,Arch Linux (UKM Edition),general,None,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
90,Ututo,"Diego Saravia, Daniel Olivera",UTUTO dev team,2000,XS 2012,?,2012-04-27,100% Free: GNU FSDG[8],"Ututo XS: Gentoo Linux, Ututo UL: Ubuntu",general,None,Inactive
91,VectorLinux,Robert S. Lange,dev team,1999,VL 7.2[102],?,2017-08-28,X,Slackware,desktop,None,Inactive
92,Void Linux,Juan Romero Pardines,Juan RP and contributors,2008,Rolling,Rolling,2022-10-17,X,"Independent, partially inspired by FreeBSD/NetBSD",general,None,Active
93,Webconverger,Kai Hendry,Webconverger Limited,2007,35.0[103],?,2016-05-19,X,Debian,"kiosk software, digital signage",Commercial,Inactive
